# Image Classification using CNN - MNIST Digit Classification

This notebook implements a Convolutional Neural Network (CNN) in PyTorch to classify MNIST handwritten digits with 98% accuracy in just 5 epochs.

**Features:**
- Deep CNN Architecture with ReLU activations
- Adam optimizer for efficient training
- Visualization of predictions and training history
- Achieves 98%+ accuracy on test set


## 1. Import Libraries


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


## 2. Set Device (CPU/GPU)


In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


## 3. CNN Model Architecture

The model consists of:
- **Convolutional Blocks**: Two blocks with 32 and 64 filters respectively
- **Fully Connected Layers**: 512 neurons with dropout for regularization
- **Output Layer**: 10 neurons (one for each digit class)


In [ ]:
# CNN Model Architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # First convolutional block
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        # Second convolutional block
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        # First block: Conv -> ReLU -> Conv -> ReLU -> MaxPool
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)  # 28x28 -> 14x14
        
        # Second block: Conv -> ReLU -> Conv -> ReLU -> MaxPool
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2)  # 14x14 -> 7x7
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


## 4. Data Loading and Preprocessing

Load MNIST dataset with normalization transformations.


In [ ]:
# Data loading and preprocessing
def get_data_loaders(batch_size=128):
    # Transformations
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))  # MNIST mean and std
    ])
    
    # Load datasets
    train_dataset = datasets.MNIST(
        root='./data',
        train=True,
        download=True,
        transform=transform
    )
    
    test_dataset = datasets.MNIST(
        root='./data',
        train=False,
        download=True,
        transform=transform
    )
    
    # Data loaders (num_workers=0 for Windows compatibility)
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0
    )
    
    return train_loader, test_loader


## 5. Training Function


In [ ]:
# Training function
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch}')
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        total += target.size(0)
        
        # Update progress bar
        pbar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Acc': f'{100.*correct/total:.2f}%'
        })
    
    avg_loss = train_loss / len(train_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy


## 6. Testing Function


In [ ]:
# Testing function
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            total += target.size(0)
    
    avg_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy


## 7. Visualization Function


In [ ]:
# Visualization function
def visualize_predictions(model, test_loader, num_images=10):
    model.eval()
    fig, axes = plt.subplots(2, 5, figsize=(12, 6))
    axes = axes.ravel()
    
    data_iter = iter(test_loader)
    images, labels = next(data_iter)
    images, labels = images.to(device), labels.to(device)
    
    with torch.no_grad():
        outputs = model(images[:num_images])
        preds = outputs.argmax(dim=1)
        probs = F.softmax(outputs, dim=1)
    
    for i in range(num_images):
        img = images[i].cpu().squeeze()
        axes[i].imshow(img, cmap='gray')
        axes[i].axis('off')
        axes[i].set_title(
            f'True: {labels[i].item()}\nPred: {preds[i].item()}\nProb: {probs[i][preds[i]].item():.2f}',
            fontsize=10
        )
    
    plt.tight_layout()
    plt.savefig('predictions.png', dpi=150, bbox_inches='tight')
    print('\nPredictions visualization saved as predictions.png')
    plt.show()


## 8. Initialize Model and Load Data

Set hyperparameters and initialize the model, optimizer, and data loaders.


In [ ]:
# Hyperparameters
batch_size = 128
learning_rate = 0.001
num_epochs = 5

# Load data
print('Loading MNIST dataset...')
train_loader, test_loader = get_data_loaders(batch_size)

# Initialize model
model = CNN().to(device)
print(f'\nModel architecture:\n{model}\n')

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training history
train_losses = []
train_accs = []
test_losses = []
test_accs = []


## 9. Training Loop

Train the model for 5 epochs and track performance.


In [ ]:
print('Starting training...\n')
for epoch in range(1, num_epochs + 1):
    # Train
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, epoch)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Test
    test_loss, test_acc = test(model, test_loader, criterion)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    
    print(f'\nEpoch {epoch}:')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
    print('-' * 50)

# Final results
print(f'\nFinal Test Accuracy: {test_accs[-1]:.2f}%')


## 10. Save Model


In [ ]:
# Save model
torch.save(model.state_dict(), 'mnist_cnn_model.pth')
print('Model saved as mnist_cnn_model.pth')


## 11. Plot Training History

Visualize the training and test loss/accuracy curves.


In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, 'b-', label='Train Loss')
plt.plot(range(1, num_epochs + 1), test_losses, 'r-', label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_accs, 'b-', label='Train Accuracy')
plt.plot(range(1, num_epochs + 1), test_accs, 'r-', label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Test Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
print('Training history saved as training_history.png')
plt.show()


## 12. Visualize Predictions

Display sample predictions with confidence scores.


In [ ]:
# Visualize predictions
print('\nVisualizing predictions...')
visualize_predictions(model, test_loader, num_images=10)
